## Huffman Coding Implementation

Based on: https://en.wikipedia.org/wiki/Huffman_coding

1. Create a leaf node for each symbol and add it to the priority queue.
1. While there is more than one node in the queue:
   * Remove the two nodes of highest priority (lowest probability) from the queue
   * Create a new internal node with these two nodes as children and with probability equal to the sum of the two nodes' probabilities.
   * Add the new node to the queue.
1. The remaining node is the root node and the tree is complete.

In [1]:
# nltk setup, run once
#import nltk
#nltk.download('punkt')

from typing import Dict # for type hints 
from nltk.tokenize import word_tokenize

In [2]:
def build_tree(probs:Dict[str,float], verbose:bool=True)->float:
    """Build binary tree following Huffmann encoding algorithm"""
    probs = probs.copy()

    # initilize priority queue - lowest probability first
    priority_queue = sorted(probs.keys(), key=probs.get)

    # iteratively add new nodes by combining 2 nodes with lowest probability
    while len(priority_queue) > 1:
        item1 = priority_queue.pop(0)
        item2 = priority_queue.pop(0)
        new_node = (item1, item2)
        probs[new_node] = probs[item1] + probs[item2]

        if verbose:
            print(f"Creating new node: P({new_node}) = {probs[new_node]:.1%}")

        # add to queue and maintain priority order: lowest prob first
        priority_queue.append(new_node)
        priority_queue.sort(key=probs.get)

    # when 1 node remains: output as result tree
    output_tree = priority_queue[0]
    return output_tree


def get_coding(node:tuple, code_prefix:str="")->Dict[str,str]:
    """Recursively translate tree output by build_tree() to 0/1 prefix code"""

    if isinstance(node, str): # end of recursion: found leaf node
        return {node: code_prefix}
    else:
        left_node, right_node = node
        left_codes = get_coding(left_node, code_prefix + "0")
        right_codes = get_coding(right_node, code_prefix + "1")
        return {**left_codes, **right_codes}


def do_huffman_encoding(probs:Dict[str,float], verbose=True)->Dict[str,str]:
    """Apply Huffman encoding steps to input probability dictionary"""
    
    if verbose==True:  # show input and output
        print("Input probs:")
        for sym, prob_sym in sorted(probs.items()):
            print(f"{sym}: {prob_sym:.4g}")
    
    # Step 1: build binary tree
    output_tree = build_tree(probs, verbose=verbose)

    # Step 2: get prefix codes
    output_coding = get_coding(output_tree)

    if verbose==True:  # show input and output            
        print("\nTree:", output_tree)
        print("\nEncoding:")    
        for sym, sym_code in sorted(output_coding.items()):
            print(f"{sym}: {sym_code}")

    return output_coding

## A fair coin

In [3]:
events = "ab"
symbol_probs = {sym:events.count(sym)/len(events) for sym in events}

code_map = do_huffman_encoding(symbol_probs)

expected_encoding_len = sum([symbol_probs[sym]*len(code_map[sym]) for sym in symbol_probs])
print("\nAverage encoded length:",expected_encoding_len)


Input probs:
a: 0.5
b: 0.5
Creating new node: P(('a', 'b')) = 100.0%

Tree: ('a', 'b')

Encoding:
a: 0
b: 1

Average encoded length: 1.0


## A fair 6-sided die

In [4]:
events = "123456"
symbol_probs = {sym:events.count(sym)/len(events) for sym in events}

code_map = do_huffman_encoding(symbol_probs)

expected_encoding_len = sum([symbol_probs[sym]*len(code_map[sym]) for sym in symbol_probs])
print(f"Average encoded length: {expected_encoding_len:.4f}")

Input probs:
1: 0.1667
2: 0.1667
3: 0.1667
4: 0.1667
5: 0.1667
6: 0.1667
Creating new node: P(('1', '2')) = 33.3%
Creating new node: P(('3', '4')) = 33.3%
Creating new node: P(('5', '6')) = 33.3%
Creating new node: P((('1', '2'), ('3', '4'))) = 66.7%
Creating new node: P((('5', '6'), (('1', '2'), ('3', '4')))) = 100.0%

Tree: (('5', '6'), (('1', '2'), ('3', '4')))

Encoding:
1: 100
2: 101
3: 110
4: 111
5: 00
6: 01
Average encoded length: 2.6667


## An unfair 6-sided die

In [5]:
events = "1111123456"
symbol_probs = {sym:events.count(sym)/len(events) for sym in events}

code_map = do_huffman_encoding(symbol_probs)

expected_encoding_len = sum([symbol_probs[sym]*len(code_map[sym]) for sym in symbol_probs])
print(f"Average encoded length: {expected_encoding_len:.4f}")

Input probs:
1: 0.5
2: 0.1
3: 0.1
4: 0.1
5: 0.1
6: 0.1
Creating new node: P(('2', '3')) = 20.0%
Creating new node: P(('4', '5')) = 20.0%
Creating new node: P(('6', ('2', '3'))) = 30.0%
Creating new node: P((('4', '5'), ('6', ('2', '3')))) = 50.0%
Creating new node: P(('1', (('4', '5'), ('6', ('2', '3'))))) = 100.0%

Tree: ('1', (('4', '5'), ('6', ('2', '3'))))

Encoding:
1: 0
2: 1110
3: 1111
4: 100
5: 101
6: 110
Average encoded length: 2.2000


## A snippet of English text

Here symbols are **characters** (not whole words).


Note that symbol probability is **not** uniformly distributed; symbol "e" is most frequent character (CORRECTION: "e' is most frequent *vowel*, "s" and "t" are the most frequent letters)

In [6]:
text="It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of foolishness"
events = list(text.lower()) # convert text to lower-case

symbol_probs = {sym:events.count(sym)/len(events) for sym in events if sym != " "}

code_map = do_huffman_encoding(symbol_probs)

expected_encoding_len = sum([symbol_probs[sym]*len(code_map[sym]) for sym in symbol_probs])
print(f"Average encoded length: {expected_encoding_len:.4f}")

Input probs:
,: 0.02778
a: 0.05556
b: 0.009259
d: 0.009259
e: 0.09259
f: 0.0463
g: 0.01852
h: 0.0463
i: 0.07407
l: 0.009259
m: 0.02778
n: 0.009259
o: 0.07407
r: 0.009259
s: 0.1111
t: 0.1111
w: 0.05556
Creating new node: P(('b', 'r')) = 1.9%
Creating new node: P(('d', 'l')) = 1.9%
Creating new node: P(('n', 'g')) = 2.8%
Creating new node: P((('b', 'r'), ('d', 'l'))) = 3.7%
Creating new node: P(('m', ',')) = 5.6%
Creating new node: P((('n', 'g'), (('b', 'r'), ('d', 'l')))) = 6.5%
Creating new node: P(('h', 'f')) = 9.3%
Creating new node: P(('w', 'a')) = 11.1%
Creating new node: P((('m', ','), (('n', 'g'), (('b', 'r'), ('d', 'l'))))) = 12.0%
Creating new node: P(('i', 'o')) = 14.8%
Creating new node: P(('e', ('h', 'f'))) = 18.5%
Creating new node: P(('t', 's')) = 22.2%
Creating new node: P((('w', 'a'), (('m', ','), (('n', 'g'), (('b', 'r'), ('d', 'l')))))) = 23.1%
Creating new node: P((('i', 'o'), ('e', ('h', 'f')))) = 33.3%
Creating new node: P((('t', 's'), (('w', 'a'), (('m', ','), (('n

## The text of Lord of the Rings 

This example builds an encoding based on the first 10k tokens in LOTR

Here are symbols are **words** and not **characters**

In [7]:
# download text of "Lord of the Rings" to current working directory
!wget https://raw.githubusercontent.com/wess/iotr/master/lotr.txt

# show first 10 lines of file:
!head lotr.txt

--2023-08-28 18:19:36--  https://raw.githubusercontent.com/wess/iotr/master/lotr.txt
Resolving raw.githubusercontent.com... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3262595 (3.1M) [text/plain]
Saving to: ‘lotr.txt.3’

lotr.txt.3          100%[===================>]   3.11M  17.6MB/s    in 0.2s    

2023-08-28 18:19:37 (17.6 MB/s) - ‘lotr.txt.3’ saved [3262595/3262595]

####-SPECIAL NOTE: 

 
In this reprint several minor inaccuracies, most of them noted by readers, have 
been corrected. For example, the rune text now corresponds exactly with the runes 
on  Thror's  Map.  More  important  is  the  matter  of  Chapter  Five.  There  the  true 
story  of  the  ending  of  the  Riddle  Game,  as  it  was  eventually  revealed  (under 
pressure)  by Bilbo  to Gandalf,  is  now  given  according  to  the Red Book,  in  place 
of  the  version  Bilbo  

In [8]:
# open file and tokenize
with open("lotr.txt") as f:
    lotr=f.read()

lotr_toks = word_tokenize(lotr.lower())

print(lotr_toks[:20])

['#', '#', '#', '#', '-special', 'note', ':', 'in', 'this', 'reprint', 'several', 'minor', 'inaccuracies', ',', 'most', 'of', 'them', 'noted', 'by', 'readers']


In [9]:
FIRST_N_TOKENS = 10000

events = lotr_toks[:FIRST_N_TOKENS]
symbol_probs = {tok:events.count(tok)/len(events) for tok in events}

# using 'verbose=False' to surpress cell output for this large dataset
code_map = do_huffman_encoding(symbol_probs, verbose=False)

In [10]:
expected_encoding_len = sum([symbol_probs[tok]*len(code_map[tok]) for tok in symbol_probs])
print(f"Average encoded length: {expected_encoding_len:.4f}")

Average encoded length: 8.4128


In [11]:
# short codes:
for symbol, code in code_map.items():
    if len(code)<=5:
        print(symbol, code)

the 0001
, 0111
. 10110
and 11101


In [12]:
# long codes - showing first 20

SHOW_N = 20

all_long_codes = [(symbol,code) for symbol, code in code_map.items() if len(code)>=13]

for symbol, code in all_long_codes[:SHOW_N]:
    print(symbol, code)

-special 0100001110000
reprint 0100001110001
minor 0100001110010
inaccuracies 0100001110011
noted 0100001110100
readers 0100001110101
corrected 0100001110110
example 0100001110111
text 0100001111000
corresponds 0100001111001
chapter 0100001111010
ending 0100001111011
riddle 0100001111100
eventually 0100001111101
revealed 0100001111110
pressure 0100001111111
according 0100010000000
version 0100010000001
actually 0100010000010
diary 0100010000011
